# 🏋️‍♂️ zure AI Foundry를 사용한 피트니스 및 건강을 위한 멀티 에이전트 RAG 🥑

재미있는 셀프 가이드 워크샵에 오신 것을 환영합니다. AutoGen 0.4.7과 Azure AI Foundry를 사용하여 다중 에이전트 검색 증강 생성(RAG) 파이프라인을 빌드합니다. 에이전트 팀이 협력하여 피트니스 및 건강 관련 질문에 여러가지 방식으로 답변해 드립니다!

## 1. 설정

필요한 라이브러리를 가져오고 Azure AI Foundry를 사용하여 모델 클라이언트를 설정해 보겠습니다. 환경 변수 `GITHUB_TOKEN`이 개인 액세스 토큰으로 설정되어 있는지 확인합니다.

In [ ]:
import os
import asyncio

# Import AutoGen agents and required modules
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Import the Azure AI Foundry model client from AutoGen extensions
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_core.models import UserMessage
from azure.core.credentials import AzureKeyCredential

# Create the model client using Azure AI Foundry
model_client = AzureAIChatCompletionClient(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": False,
        "function_calling": False,
        "vision": False,
        "family": "unknown"
    }
)

print("✅ Azure AI Foundry model client created successfully!")

## 2. 샘플 건강 데이터 및 검색 도구 만들기

간단한 건강 팁 목록과 간단한 검색 기능을 정의하겠습니다. 이 함수는 사용자 쿼리의 키워드를 기반으로 관련 건강 팁을 검색하는 것을 시뮬레이션합니다.

In [ ]:
# Define sample health tips
health_tips = [
    {"id": "tip1", "content": "Do a 10-minute HIIT workout to boost your metabolism.", "source": "Fitness Guru"},
    {"id": "tip2", "content": "Take a brisk 15-minute walk to clear your mind and improve circulation.", "source": "Health Coach"},
    {"id": "tip3", "content": "Stretch for 5 minutes every hour if you're sitting at a desk.", "source": "Wellness Expert"},
    {"id": "tip4", "content": "Incorporate strength training twice a week for overall fitness.", "source": "Personal Trainer"},
    {"id": "tip5", "content": "Drink water regularly to stay hydrated during workouts.", "source": "Nutritionist"}
]

def retrieve_tips(query: str) -> str:
    """Return health tips whose content contains keywords from the query."""
    query_lower = query.lower()
    relevant = []
    for tip in health_tips:
        # Check if any word in the query is in the tip content
        if any(word in tip["content"].lower() for word in query_lower.split()):
            relevant.append(f"Source: {tip['source']} => {tip['content']}")
    if not relevant:
        # If no tips match, return all tips (for demo purposes)
        relevant = [f"Source: {tip['source']} => {tip['content']}" for tip in health_tips]
    return "\n".join(relevant)

print("✅ Sample health tips and retrieval tool created!")

## 3. 멀티 에이전트 RAG 파이프라인 정의하기

두 개의 에이전트를 생성합니다:

1. **RetrieverAgent**: 사용자의 쿼리를 기반으로 관련 피트니스 및 건강 팁을 가져오기 위해 `retrieve_tips` 도구를 사용합니다.
2. **ResponderAgent**: 검색된 팁을 사용자의 쿼리와 함께 사용하여 재미있고 매력적인 응답을 작성합니다.

이러한 에이전트는 공동 문제 해결을 시뮬레이션하기 위해 RoundRobinGroupChat에 결합됩니다.

In [ ]:
# Define the RetrieverAgent with the retrieve_tips tool
retriever = AssistantAgent(
    name="RetrieverAgent",
    system_message=(
        "You are a smart retrieval agent. Your task is to fetch relevant fitness and health tips based on the user's query. "
        "Use the provided tool 'retrieve_tips' to get the information."
    ),
    model_client=model_client,
    tools=[retrieve_tips],  # Register the retrieval tool
    reflect_on_tool_use=True
)

# Define the ResponderAgent which uses the retrieved tips to craft a response
responder = AssistantAgent(
    name="ResponderAgent",
    system_message=(
        "You are a friendly fitness coach. Use the health tips provided to craft a fun and engaging response to the user's question."
    ),
    model_client=model_client
)

# Combine the two agents in a RoundRobinGroupChat
group_chat = RoundRobinGroupChat(
    agents=[retriever, responder],
    termination_condition=None,  # For simplicity, we run a fixed number of turns
    max_turns=4  # Limit the conversation to 4 turns
)

print("✅ Multi-agent RAG pipeline defined!")

## 4. 쿼리 시도하기

재미있는 피트니스 쿼리로 멀티 에이전트 RAG 시스템을 테스트해 보겠습니다. 예를 들어:

> **User Query:** _I'm very busy but want to stay fit. What quick exercises can I do?_

In [ ]:
async def run_query():
    user_query = "I'm very busy but want to stay fit. What quick exercises can I do?"

    # Run the group chat to process the query
    final_result = None
    async for message in group_chat.run_stream(task=user_query):
        # Print each message as it is produced
        if hasattr(message, 'content'):
            print(f"{message.source}: {message.content}")
        final_result = message
    
    print("\n✅ Final response:", final_result.content if final_result else "No response")

asyncio.run(run_query())

## 5. 결론

이 노트북에서는 Azure AI Foundry와 피트니스 및 건강 테마가 포함된 AutoGen 0.4.7을 사용하여 재미있는 멀티 에이전트 검색-증강 생성 파이프라인을 구축했습니다. 관련 건강 정보를 가져오는 검색 에이전트와 사용자의 쿼리에 대한 매력적인 답변을 작성하는 응답자 에이전트를 만들었습니다.

이 노트북을 자유롭게 수정하고 확장하여 더 발전된 다중 에이전트 협업을 탐색해 보세요. 즐겁게 코딩하시고, 건강에 유의하세요! 💪🥦